# Open-Pit Mine Problem


***

# Dataset Zuck_small
### Blocks file

In [2]:
import pandas as pd

Zuck_small = "https://drive.google.com/uc?export=download&id=1OlBuzks5S4DFQC5PRyngfddEpE-5lKUQ"

blockmodel = pd.read_csv(Zuck_small, sep=',')

blockmodel['id'] = range(1, len(blockmodel) + 1)
blockmodel['blockvalue'] = blockmodel['value'] - blockmodel['cost']
blockmodel = blockmodel.rename(columns={'x': 'xi', 'y': 'yi', 'z': 'zi'})

blockmodel

,xi,yi,zi,cost,value,rock_tonnes,ore_tonnes,id,blockvalue
0,0,0,0,0.0,0.0,0.0,0.0,1,0.0
1,0,0,1,0.0,0.0,0.0,0.0,2,0.0
2,0,0,2,0.0,0.0,0.0,0.0,3,0.0
3,0,0,3,0.0,0.0,0.0,0.0,4,0.0
4,0,0,4,0.0,0.0,0.0,0.0,5,0.0
...,...,...,...,...,...,...,...,...,...
24552,38,42,10,0.0,0.0,0.0,0.0,24553,0.0
24553,38,42,11,0.0,0.0,0.0,0.0,24554,0.0
24554,38,42,12,0.0,0.0,0.0,0.0,24555,0.0
24555,38,42,13,0.0,0.0,0.0,0.0,24556,0.0


### Conversão Coordenada Índice

In [3]:
def coord2index(df, O, D, cols=['xi', 'yi', 'zi']):
    data = [D[i]*(df[cols].iloc[:, i] - 0.5) + O[i] for i in range(3)]
    return pd.DataFrame(data, index = ['x', 'y', 'z']).T

D = [30, 30, 30]  
O = [0, 0, 0]           

dados_em_index = coord2index(blockmodel, O, D)
blockmodel  = pd.concat([dados_em_index, blockmodel], axis = 1).round(2)

colunas_ordenadas = ['id','x', 'y', 'z', 'xi', 'yi', 'zi','value', 'cost','blockvalue', 'rock_tonnes', 'ore_tonnes']
blockmodel = blockmodel[colunas_ordenadas]

blockmodel.to_csv("Zuck_small.blocks.csv", index=False)

blockmodel

,id,x,y,z,xi,yi,zi,value,cost,blockvalue,rock_tonnes,ore_tonnes
0,1,-15.0,-15.0,-15.0,0,0,0,0.0,0.0,0.0,0.0,0.0
1,2,-15.0,-15.0,15.0,0,0,1,0.0,0.0,0.0,0.0,0.0
2,3,-15.0,-15.0,45.0,0,0,2,0.0,0.0,0.0,0.0,0.0
3,4,-15.0,-15.0,75.0,0,0,3,0.0,0.0,0.0,0.0,0.0
4,5,-15.0,-15.0,105.0,0,0,4,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
24552,24553,1125.0,1245.0,285.0,38,42,10,0.0,0.0,0.0,0.0,0.0
24553,24554,1125.0,1245.0,315.0,38,42,11,0.0,0.0,0.0,0.0,0.0
24554,24555,1125.0,1245.0,345.0,38,42,12,0.0,0.0,0.0,0.0,0.0
24555,24556,1125.0,1245.0,375.0,38,42,13,0.0,0.0,0.0,0.0,0.0


### Plotting the content graph caixote

In [9]:
import pyvista as pv
import numpy as np
import matplotlib.pyplot as plt

# Filtra blocos com valor válido
df_filtrado = blockmodel.dropna(subset=['blockvalue']).copy()

# Tamanho dos blocos atualizado
dx, dy, dz = 30.0, 30.0, 30.0

# Normaliza os valores para aplicar colormap
min_val = df_filtrado['blockvalue'].min()
max_val = df_filtrado['blockvalue'].max()
valores_normalizados = (df_filtrado['blockvalue'] - min_val) / (max_val - min_val)

# Paleta Turbo
cmap = plt.get_cmap('turbo')

# Inicializa listas para malha
points = []
cells = []
cell_types = []
valores = []

point_id = 0

for i, row in df_filtrado.iterrows():
    x, y, z = row['x'], row['y'], row['z']
    valor = row['blockvalue']

    # Vértices do bloco
    x0, x1 = x - dx / 2, x + dx / 2
    y0, y1 = y - dy / 2, y + dy / 2
    z0, z1 = z - dz / 2, z + dz / 2

    bloco_vertices = [
        [x0, y0, z0],
        [x1, y0, z0],
        [x1, y1, z0],
        [x0, y1, z0],
        [x0, y0, z1],
        [x1, y0, z1],
        [x1, y1, z1],
        [x0, y1, z1],
    ]

    points.extend(bloco_vertices)
    cells.append([8] + list(range(point_id, point_id + 8)))
    cell_types.append(pv.CellType.HEXAHEDRON)
    valores.append(valor)
    point_id += 8

# Converte para arrays
points = np.array(points)
cells = np.hstack(cells)

# Cria UnstructuredGrid
grid = pv.UnstructuredGrid(cells, cell_types, points)
grid.cell_data['blockvalue'] = np.array(valores)

# Cria plotter
plotter = pv.Plotter(window_size=(1000, 700))
plotter.add_mesh(grid, scalars='blockvalue', cmap='turbo', show_edges=False)
plotter.add_scalar_bar(title='blockvalue', n_labels=5)
plotter.add_axes()
plotter.add_text(f'Caixote', position='upper_right', font_size=14)

# Exibe
plotter.show()


Widget(value='<iframe src="http://localhost:61304/index.html?ui=P_0x2714af9f410_3&reconnect=auto" class="pyvis…

### Plotting the mineral body content graph

In [12]:
import pyvista as pv
import numpy as np
import matplotlib.pyplot as plt

# Filtra blocos com valor válido
df_filtrado = blockmodel[blockmodel['blockvalue'] > 0]

# Tamanho dos blocos atualizado
dx, dy, dz = 30.0, 30.0, 30.0

# Normaliza os valores para aplicar colormap
min_val = df_filtrado['blockvalue'].min()
max_val = df_filtrado['blockvalue'].max()
valores_normalizados = (df_filtrado['blockvalue'] - min_val) / (max_val - min_val)

# Paleta Turbo
cmap = plt.get_cmap('turbo')

# Inicializa listas para malha
points = []
cells = []
cell_types = []
valores = []

point_id = 0

for i, row in df_filtrado.iterrows():
    x, y, z = row['x'], row['y'], row['z']
    valor = row['blockvalue']

    # Vértices do bloco
    x0, x1 = x - dx / 2, x + dx / 2
    y0, y1 = y - dy / 2, y + dy / 2
    z0, z1 = z - dz / 2, z + dz / 2

    bloco_vertices = [
        [x0, y0, z0],
        [x1, y0, z0],
        [x1, y1, z0],
        [x0, y1, z0],
        [x0, y0, z1],
        [x1, y0, z1],
        [x1, y1, z1],
        [x0, y1, z1],
    ]

    points.extend(bloco_vertices)
    cells.append([8] + list(range(point_id, point_id + 8)))
    cell_types.append(pv.CellType.HEXAHEDRON)
    valores.append(valor)
    point_id += 8

# Converte para arrays
points = np.array(points)
cells = np.hstack(cells)

# Cria UnstructuredGrid
grid = pv.UnstructuredGrid(cells, cell_types, points)
grid.cell_data['blockvalue'] = np.array(valores)

# Cria plotter
plotter = pv.Plotter(window_size=(1000, 700))
plotter.add_mesh(grid, scalars='blockvalue', cmap='turbo', show_edges=False)
plotter.add_scalar_bar(title='blockvalue', n_labels=5)
plotter.add_axes()
plotter.add_text(f'Mineral', position='upper_right', font_size=14)

# Exibe
plotter.show()


Widget(value='<iframe src="http://localhost:61304/index.html?ui=P_0x2714980b190_6&reconnect=auto" class="pyvis…